<a href="https://colab.research.google.com/github/hyejwon/1_ne/blob/main/TF_hub_%EB%A1%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#케라스와 텐서플로 허브를 사용한 영화 리뷰 텍스트 분류하기
 

In [ ]:
import numpy as np
import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("버전:", tf.__version__)
print("즉시 실행 모드", tf.executing_eagerly())
print("허브 버전",hub.__version__)
print("GPU","사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능")

In [ ]:
#IMDB 데이터셋 다운로드
##훈련 세트를 6대 4로 나눕니다.
##결국 훈련에 15,000개 샘플, 검증에 10,000개 샘플,테스트에 25,000개 샘플을 사용 
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]','train[60%:]','test'),
    as_supervised=True)

In [ ]:
#데이터 탐색
##데이터셋의 샘플은 전처리된 정수 배열.이 정수는 영화 리뷰에 나오는 단어를 나타냄
###레이블(label)은 정수 0 또는 1/ 0은 부정적인 리뷰이고 1은 긍정적인 리뷰

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

In [ ]:
train_labels_batch

In [ ]:
#모델 구성 
#텍스트를 표현하는 한 가지 방법은 문장을 임베딩 벡ㅌㅓ로 바꾸는 것
#첫 번째 층으로 사전 훈련된(pre-trained)된 텍스트 임베딩을 사용할 수 있음
###장점: 텍스트 전처리에 대해 신경 쓸 필요가 없다
###     전이 학습의 장점을 이용
###     임베딩은 고정크기이기 때문에 처리 과정이 단순
#이 예제는 텐서플로 허브에 있는 사전 훈련된 텍스트 임베딩 모델 사용
# 먼저 문장을 임베딩 시키기 위해 텐서플로 허브 모델을 사용하는 케라스 층을 만듦
# 그 다음 몇 개의 샘플을 입력하여 테스트. 입력 텍스트의 길이에 상관없이 임베딩의 출력 크기는 (num_examples,embedding_dimension)

embedding ="http://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                            dtype=tf.string,trainable=True)
hub_layer(train_examples_batch[:3])

In [ ]:
#전체 모델
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()


In [ ]:
#손실 함수와 옵티마이저
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#모델 훈련
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

In [ ]:
#모델 평가

results = model.evaluate(test_data.batch(512),verbose=2)

for name, value in zip(model.metrics_names,results):
    print("%s:%.3f"%(name,value))